In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
from torchvision import transforms, datasets
import torchvision
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import time

In [2]:
class DarkNet(nn.Module):
    def __init__(self):

        super(DarkNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.LeakyReLU(0.1)

        self.pool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.LeakyReLU(0.1)

        self.pool2 = nn.MaxPool2d(2)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.LeakyReLU(0.1)

        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm2d(64)
        self.relu4 = nn.LeakyReLU(0.1)

        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn5 = nn.BatchNorm2d(128)
        self.relu5 = nn.LeakyReLU(0.1)

        self.pool5 = nn.MaxPool2d(2)

        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn6 = nn.BatchNorm2d(256)
        self.relu6 = nn.LeakyReLU(0.1)

        self.conv7 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn7 = nn.BatchNorm2d(128)
        self.relu7 = nn.LeakyReLU(0.1)

        self.conv8 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn8 = nn.BatchNorm2d(256)
        self.relu8 = nn.LeakyReLU(0.1)

        self.pool8 = nn.MaxPool2d(2)

        self.conv9 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn9 = nn.BatchNorm2d(512)
        self.relu9 = nn.LeakyReLU(0.1)

        self.conv10 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn10 = nn.BatchNorm2d(256)
        self.relu10 = nn.LeakyReLU(0.1)

        self.conv11 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn11 = nn.BatchNorm2d(512)
        self.relu11 = nn.LeakyReLU(0.1)

        self.conv12 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn12 = nn.BatchNorm2d(256)
        self.relu12 = nn.LeakyReLU(0.1)

        self.conv13 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn13 = nn.BatchNorm2d(512)
        self.relu13 = nn.LeakyReLU(0.1)

        self.pool13 = nn.MaxPool2d(2)

        self.conv14 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn14 = nn.BatchNorm2d(1024)
        self.relu14 = nn.LeakyReLU(0.1)

        self.conv15 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn15 = nn.BatchNorm2d(512)
        self.relu15 = nn.LeakyReLU(0.1)

        self.conv16 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn16 = nn.BatchNorm2d(1024)
        self.relu16 = nn.LeakyReLU(0.1)

        self.conv17 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=1, bias=False)
        self.bn17 = nn.BatchNorm2d(512)
        self.relu17 = nn.LeakyReLU(0.1)
        
        self.conv18 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn18 = nn.BatchNorm2d(1024)
        self.relu18 = nn.LeakyReLU(0.1)

        self.conv19 = nn.Conv2d(in_channels=1024, out_channels=30, kernel_size=3, stride=1, padding=1, bias=False)

        self.pool19 = nn.AvgPool2d(7)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)
        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)
        x = self.pool5(x)
        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu6(x)
        x = self.conv7(x)
        x = self.bn7(x)
        x = self.relu7(x)
        x = self.conv8(x)
        x = self.bn8(x)
        x = self.relu8(x)
        x = self.pool8(x)
        x = self.conv9(x)
        x = self.bn9(x)
        x = self.relu9(x)
        x = self.conv10(x)
        x = self.bn10(x)
        x = self.relu10(x)
        x = self.conv11(x)
        x = self.bn11(x)
        x = self.relu11(x)
        x = self.conv12(x)
        x = self.bn12(x)
        x = self.relu12(x)
        x = self.conv13(x)
        x = self.bn13(x)
        x = self.relu13(x)
        x = self.pool13(x)
        x = self.conv14(x)
        x = self.bn14(x)
        x = self.relu14(x)
        x = self.conv15(x)
        x = self.bn15(x)
        x = self.relu15(x)
        x = self.conv16(x)
        x = self.bn16(x)
        x = self.relu16(x)
        x = self.conv17(x)
        x = self.bn17(x)
        x = self.relu17(x)
        x = self.conv18(x)
        x = self.conv19(x)
        x = self.pool19(x)
        x = x.view(x.size(0), -1)
        return x

In [3]:
batchsize = 32
train_data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(10),
        transforms.ColorJitter(hue=0.1,saturation=0.75),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

tset_data_transform = transforms.Compose([
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

train_dataset = torchvision.datasets.ImageFolder(root='/media/lulugay/PC/CCCV-30/train_set',transform=train_data_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize, shuffle=True, num_workers=8)
 
val_dataset = torchvision.datasets.ImageFolder(root='/media/lulugay/PC/CCCV-30/test_set', transform=tset_data_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2*batchsize, shuffle=True, num_workers=8)


In [4]:
model = DarkNet()
model.cuda()
loss_func = nn.CrossEntropyLoss()

In [5]:
def test(net,data_loader):
    corret,total = 0,0
    for inputs,labels in data_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(Variable(inputs))
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        corret += (predicted == labels).sum()
    print('Accuracy of the network on the test images: %d %%' % (100 * corret / total))

def poly(base_lr,power,total_epoch,now_epoch):
    return base_lr*(1-math.pow(float(now_epoch)/float(total_epoch),power))

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    batch_size_start = time.time()
    running_loss = 0.0
    corret,total = 0,0
    print('train epoch%d'%epoch)
    for i,(inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        inputs = Variable(inputs)
        lables = Variable(labels)
        lr=poly(0.01,4,num_epochs,epoch)
        optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)
        optimizer.zero_grad()
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        corret += (predicted == labels).sum()
        loss = loss_func(outputs, labels)        #交叉熵
        loss.backward()
        optimizer.step()                          #更新权重
        running_loss += loss.data[0]
    print('accuracy on train set%d %%'%(100 * corret / total)) 
    print('test epoch%d'%epoch)
    test(model,val_loader)
    torch.save(model.state_dict(),'epoch%d_weight.pkl'%epoch)
    print('Epoch [%d/%d], Loss: %.4f,need time %.4f' % (epoch + 1, num_epochs, running_loss / (45000 / batchsize), time.time() - batch_size_start))
    

train epoch0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:22: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


accuracy on train set18 %
test epoch0
Accuracy of the network on the test images: 24 %
Epoch [1/100], Loss: 2.8371,need time 1224.3933
train epoch1
accuracy on train set29 %
test epoch1
Accuracy of the network on the test images: 31 %
Epoch [2/100], Loss: 2.4003,need time 142.2772
train epoch2
accuracy on train set35 %
test epoch2
Accuracy of the network on the test images: 37 %
Epoch [3/100], Loss: 2.1755,need time 143.3074
train epoch3
accuracy on train set39 %
test epoch3
Accuracy of the network on the test images: 40 %
Epoch [4/100], Loss: 2.0327,need time 143.9551
train epoch4
accuracy on train set42 %
test epoch4
Accuracy of the network on the test images: 43 %
Epoch [5/100], Loss: 1.9115,need time 143.4011
train epoch5
accuracy on train set44 %
test epoch5
Accuracy of the network on the test images: 44 %
Epoch [6/100], Loss: 1.8256,need time 142.2933
train epoch6
accuracy on train set47 %
test epoch6
Accuracy of the network on the test images: 46 %
Epoch [7/100], Loss: 1.7384,ne